In [1]:
import cobra
import glob
import pickle
import os
import re
import json

In [2]:
# Read in model_path names
model_paths = glob.glob('/home/bneubert/Lactobacillus/gap_models/*.xml')

In [4]:
#model_paths
#model_x = cobra.io.read_sbml_model(model_paths[1])

In [5]:
# takes a list of patric_id's(list of lists, first list is by genome, second by feature) and uses the feature file 
# to pull out the pgfam_id
# outputs a dictionary with patric_ids linked to their pgfam_ids
def pgfam_finder(feature_master):
    
    output = {}
    # create blank dictionary of keys for every genomic feature in the input list
    for genome_id in feature_master:
        for id in genome_id:
            output[id] = ''

    # create regular expression to identify the genome id from the patric id of the genomic feature
    test = re.compile('(\d+)\.\d+\.')
    
    # variables used for % complete calculation
    total = len(feature_master) 
    count = 0.0
    prev = -1
    
    # find associated protein families for every genomic feature within the models
    # start cycle through first genome
    for genome_id in feature_master:
        # open feature file via associated genome through regular expression
        temp = pickle.load(open('/home/bneubert/Lactobacillus/features/'+ test.match(genome_id[1]).group()+'feats'))
        # cycle through features in this genome
        for feature_id in genome_id:
            # check every feature in the feature file for this feature to find associated protein family
            for i in range(0,len(temp)):
                
                if(('fig|'+str(feature_id)) == str(temp[i]['patric_id'])):  
                        # if this has a protein family associated with it, add this to the dict
                        if('pgfam_id' in temp[i].keys()):
                            # only ever single family values so can use dict mapping string to string
                            output[str(feature_id)] = str(temp[i]['pgfam_id'])
                        # if there is not an associated protein family, then add a blank string
                        else:
                            output[str(feature_id)] = ''
        count += 1
        # if we have changed by a percentage point then update the status
        if (int(count/total*100) > prev):
            prev = int(count/total*100)
            print("Status: "+str(prev)+"% complete finding protein families")
    
    return output

In [6]:
total_genes = []
counter = 0.0
prev = -1
total = len(model_paths)
# create a list of all the genes within the models
# list of lists, initial list by genome and inner list is full of features within that genome
for model_path in model_paths:
    model_x = cobra.io.read_sbml_model(model_path)
    temp = []
    for gene in model_x.genes:
        temp.append(gene.id)
    total_genes.append(temp)
    counter += 1
    # if the status has changed by a percentage point then update status
    if (int(counter/total*100) > prev):
        prev = int(counter/total*100)
        print("Status: "+str(prev)+"% complete finding genes present in models")
#     use this  to test a smaller portion the models
#     if( counter is 5):
#         break

# run function for genes within the models
total_families = pgfam_finder(total_genes)  

Status: 0% complete finding genes present in models
Status: 1% complete finding genes present in models
Status: 2% complete finding genes present in models
Status: 3% complete finding genes present in models
Status: 4% complete finding genes present in models
Status: 5% complete finding genes present in models
Status: 6% complete finding genes present in models
Status: 7% complete finding genes present in models
Status: 8% complete finding genes present in models
Status: 9% complete finding genes present in models
Status: 10% complete finding genes present in models
Status: 11% complete finding genes present in models
Status: 12% complete finding genes present in models
Status: 13% complete finding genes present in models
Status: 14% complete finding genes present in models
Status: 15% complete finding genes present in models
Status: 16% complete finding genes present in models
Status: 17% complete finding genes present in models
Status: 18% complete finding genes present in models
Sta

Status: 63% complete finding protein families
Status: 64% complete finding protein families
Status: 65% complete finding protein families
Status: 66% complete finding protein families
Status: 67% complete finding protein families
Status: 68% complete finding protein families
Status: 69% complete finding protein families
Status: 70% complete finding protein families
Status: 71% complete finding protein families
Status: 72% complete finding protein families
Status: 73% complete finding protein families
Status: 74% complete finding protein families
Status: 75% complete finding protein families
Status: 76% complete finding protein families
Status: 77% complete finding protein families
Status: 78% complete finding protein families
Status: 79% complete finding protein families
Status: 80% complete finding protein families
Status: 81% complete finding protein families
Status: 82% complete finding protein families
Status: 83% complete finding protein families
Status: 84% complete finding prote

In [8]:
#total_families

In [11]:
len(total_families)

811608

In [12]:
# perhaps add genome name in future?? any other worthwhile information from the original pickle objects?
counter = 0
new_total = []
test = re.compile('(\d+)\.\d+')

# for every feature in the model features list, change its format to the same as the .features files output by mackinac
# so that processing does not need to be changed 
for feature, family in total_families.items():
    temp = {}
    pattern = test.match(feature)
    if(pattern is not None ):
        temp[unicode('patric_id')] = unicode('fig|'+feature)
        temp[unicode('genome_id')] = unicode(pattern.group())
        temp[unicode('pgfam_id')] = unicode(family)
        new_total.append(temp)
    else: 
        print("Feature id: "+ str(feature)+", Family id: "+str(family))

    counter += 1
#     if (counter is 3):
#         break

Feature id: rast, Family id: 
Feature id: rast|0.CDS.1, Family id: 
Feature id: 0.CDS.1, Family id: 


In [13]:
len(new_total)

811605

In [14]:
file_name = "/home/bneubert/Lactobacillus/Data/model.features"

In [15]:
new_list = sorted(new_total,key=lambda k: k[unicode('genome_id')])

In [21]:
#new_list

In [17]:
test_model = cobra.io.read_sbml_model('/home/bneubert/Lactobacillus/gap_models/1579.60.xml')
test_model.genes

[<Gene 1579.60.peg.21 at 0x7f0d2278be10>,
 <Gene 1579.60.peg.16 at 0x7f0d2278bf10>,
 <Gene 1579.60.peg.11 at 0x7f0d2278b5d0>]

In [23]:
# identify indices with bad model to remove
to_remove = []
index = 0

for feature in new_list:
    if(feature[unicode('genome_id')] == unicode('1579.60')):
        to_remove.append(index)
    
    index += 1
to_remove

[239406, 239407, 239408]

In [24]:
# remove bad model
del new_list[239406:239409]
len(new_list)

811602

In [25]:
# double check that the bad model was removed
to_remove = []
index = 0

for feature in new_list:
    if(feature[unicode('genome_id')] == unicode('1579.60')):
        to_remove.append(index)
    
    index += 1
to_remove

[]

In [27]:
# save feature file as json with bad model removed
json.dump(new_list, open(file_name, "wb"))